chromedriver download: https://chromedriver.chromium.org/downloads (match to you chrome version)

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import numpy as np
import pandas as pd
from selenium import webdriver

In [9]:
def web_scrape_page():
    
    url = driver.current_url

    resp = requests.get(url)
    
    # All info from webpage scraped
    soup = BeautifulSoup(resp.text,'html.parser')
    
    # Scrape coa_dg_table table for all string elements
    data = []
    table = soup.find('table', attrs={'class':'coa_dg_table'})
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
        
    final_df = pd.DataFrame(data)
    final_df = final_df.rename(columns={0: "Case Name", 
                                        1: "Case No.",
                                        2: "Case Panel",
                                        3: "Hearing Location",
                                        4: "Hearing Date",
                                        5: "Audio",
                                        6: "Video"})
    
    # Scrape coa_dg_table table for links to audio and video files
    audio = []
    video = []

    for a in table_body.find_all('a', href=True):
        if 'video' in a['href']:
            video.append(a['href'])
        else:
            audio.append(a['href'])
    
    # Add parent website to beinning of string
    audio = ['https://www.ca9.uscourts.gov/media/' + x for x in audio]
    video = ['https://www.ca9.uscourts.gov/media/' + x for x in video]
    
    # Append to final dataframe
    final_df['Audio'] = audio
    final_df['Video'] = video
    
    
    return final_df

Run the following to download the correct chromedriver

In [ ]:
# !pip install seleniumbase

In [ ]:
# !seleniumbase install chromedriver latest

Take path out (ex: '/anaconda3/lib/python3.7/site-packages/seleniumbase/drivers/chromedriver') and replace line 14 below. If still doesnt work let me know.

In [13]:
from selenium.common.exceptions import NoSuchElementException
import time

#Initializing pg DataFrame that will eventually contain all scraped information
pg = pd.DataFrame()

#Using selenium's webdriver to create basic Chrome Options when opening browser
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument("--test-type")

#Using selenium's webdriver to open Google Chrome with stated options 
#Goes directly to url which is the first page of IMDB Top 250 list  
driver = webdriver.Chrome('/anaconda3/lib/python3.7/site-packages/seleniumbase/drivers/chromedriver', options = options)
driver.get("https://www.ca9.uscourts.gov/media/")

#Scrape page until there isn't a 'Next »' button
#Merge previous pages (pg) to current page (pg_current) until no pages are remaining
pages_remaining = True

while pages_remaining == True:

    pg_current = web_scrape_page()
    pg = pd.concat([pg,pg_current])

    try:
        next_link = driver.find_element_by_link_text(">>") 
        next_link.click()
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)

    except NoSuchElementException:
        pages_remaining = False

#Close browser
driver.close()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=85.0.4183.121)


In [11]:
pg

,Case Name,Case No.,Case Panel,Hearing Location,Hearing Date,Audio,Video
0,Jenny Flores v. William Barr,20-55951,"W. FLETCHER, BERZON, M. SMITH, Jr.",San Francisc...,09/30/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
1,Gary Romanchuk v. SCUF & CWUF Joint Benefit Fund,17-56069,"RAWLINSON, BENNETT, BADE",Pasadena CA,09/29/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
2,USA v. Justin Nekeferoff,19-30015,"CHRISTEN, WATFORD, BADE",Anchorage AK,09/25/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
3,"George Young, Jr. v. State of Hawaii",12-17808,"O'SCANNLAIN, THOMAS, McKEOWN, WARDLAW, W. FLET...",San Francisc...,09/24/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
4,Maria Medina Tovar v. Laura Zuchowski,18-35072,"THOMAS, McKEOWN, GRABER, RAWLINSON, CALLAHAN, ...",San Francisc...,09/23/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
5,Jorge Rojas v. FAA,17-55036,"THOMAS, WARDLAW, GRABER, RAWLINSON, CALLAHAN, ...",San Francisc...,09/22/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
6,"Harvest Rock Church, Inc. v. Gavin Newsom",20-55907,"O'SCANNLAIN, RAWLINSON, CHRISTEN",San Francisc...,09/21/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
7,USA v. Isaac Bautista,19-10448,"SCHROEDER, W. FLETCHER, HUNSAKER",San Francisc...,09/18/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
8,Alice Brown v. County of Del Norte,18-16689,"SCHROEDER, W. FLETCHER, HUNSAKER",San Francisc...,09/18/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....
9,Jeffrey Green v. City of Phoenix,19-16682,"SCHROEDER, W. FLETCHER, HUNSAKER",San Francisc...,09/18/2020,https://www.ca9.uscourts.gov/media/view.php?pk...,https://www.ca9.uscourts.gov/media/view_video....


In [5]:
#test to see if your chromedriver is working, should open google 

import time
from selenium import webdriver

driver = webdriver.Chrome('/anaconda3/lib/python3.7/site-packages/seleniumbase/drivers/chromedriver')  # Optional argument, if not specified will search path.
driver.get('http://www.google.com/');
time.sleep(5) # Let the user actually see something!
search_box = driver.find_element_by_name('q')
search_box.send_keys('ChromeDriver')
search_box.submit()
time.sleep(5) # Let the user actually see something!
driver.quit()